## Парсинг данных с сайта на Python

### Определения

Парсинг - это процесс сбора данных с последующей их обработкой и анализом.

Программа, которая занимается парсингом, называют - парсер.


### Условие задачи

С сайта ( https://habr.com/ru/search/ ) необходимо построить исходный набор данных (.csv или .xml). Набор данных должен включать __названия, описание, рейтинг и сферу деятельности компаний, дату публикации, а также текст статей из Интернет-ресурсов__. Подготовленный набор данных должен содержать сведения о всех номинантах конкурса. Разработанный парсер должен извлекать гиперссылки из начальной страницы с последующим обходом всех страниц по полученным ссылкам и извлечением их содержимого. Можно дополнить набор какими-либо другими данными, если они могут быть полезны для дальнейшего исследования.


### Этапы парсинга

1. Поиск данных
2. Получение информации
3. Сохранение данных

### Подключение библиотек

In [2]:
from bs4 import BeautifulSoup as bs

Beautiful Soup - это библиотека Python для извлечения данных из HTML и XML файлов. 

In [3]:
import requests

Библиотека requests является стандартным инструментом для составления HTTP-запросов в Python.

In [4]:
import pandas as pd

### Получение информаций

In [1]:
# GET - запрос
url = 'https://www.kinopoisk.ru/reviews/type/comment/period/month/page/1/#list' # страница со всеми статьями 
page = requests.get(url)

NameError: name 'requests' is not defined

 Метод GET указывает на то, что происходит попытка извлечь данные из определенного ресурса, в нашем случае из страницы habr.

In [6]:
page.status_code

200

Если мы введем «page.status_code», то получим статус состояния HTTP. Если статус 200, то это означает, что мы получили положительный ответ и можем дальше продолжать работу. 


Теперь получим HTML код со страницы, воспользовавшись BeautifulSoup и скормив ему page, указываем в кавычках как он нам поможет 'html.parcer':

In [7]:
soup = bs(page.text, 'html.parser')

В __page.text__ хранится html код с веб-страницы. __'html.parser'__ параметр отвечающий за действие с текстом. В __soup__ записвается спарсеный html код. 

In [8]:
page.text

'<!DOCTYPE html>\n<html lang="ru" data-vue-meta="%7B%22lang%22:%7B%22ssr%22:%22ru%22%7D%7D">\n<head >\n  <meta charset="UTF-8">\n  <meta name="viewport" content="width=device-width,initial-scale=1.0,viewport-fit=cover,maximum-scale=1,user-scalable=0">\n  <meta name="referrer" content="unsafe-url">\n  <title>Все статьи подряд / Хабр</title>\n  <style>\n    /* cyrillic-ext */\n    @font-face {\n      font-family: \'Fira Sans\';\n      font-style: normal;\n      font-weight: 500;\n      font-display: swap;\n      src: url(https://fonts.gstatic.com/s/firasans/v11/va9B4kDNxMZdWfMOD5VnZKveSxf6TF0.woff2) format(\'woff2\');\n      unicode-range: U+0460-052F, U+1C80-1C88, U+20B4, U+2DE0-2DFF, U+A640-A69F, U+FE2E-FE2F;\n    }\n\n    /* cyrillic */\n    @font-face {\n      font-family: \'Fira Sans\';\n      font-style: normal;\n      font-weight: 500;\n      font-display: swap;\n      src: url(https://fonts.gstatic.com/s/firasans/v11/va9B4kDNxMZdWfMOD5VnZKveQhf6TF0.woff2) format(\'woff2\');\n    

In [9]:
soup

<!DOCTYPE html>

<html data-vue-meta="%7B%22lang%22:%7B%22ssr%22:%22ru%22%7D%7D" lang="ru">
<head>
<meta charset="utf-8"/>
<meta content="width=device-width,initial-scale=1.0,viewport-fit=cover,maximum-scale=1,user-scalable=0" name="viewport"/>
<meta content="unsafe-url" name="referrer"/>
<title>Все статьи подряд / Хабр</title>
<style>
    /* cyrillic-ext */
    @font-face {
      font-family: 'Fira Sans';
      font-style: normal;
      font-weight: 500;
      font-display: swap;
      src: url(https://fonts.gstatic.com/s/firasans/v11/va9B4kDNxMZdWfMOD5VnZKveSxf6TF0.woff2) format('woff2');
      unicode-range: U+0460-052F, U+1C80-1C88, U+20B4, U+2DE0-2DFF, U+A640-A69F, U+FE2E-FE2F;
    }

    /* cyrillic */
    @font-face {
      font-family: 'Fira Sans';
      font-style: normal;
      font-weight: 500;
      font-display: swap;
      src: url(https://fonts.gstatic.com/s/firasans/v11/va9B4kDNxMZdWfMOD5VnZKveQhf6TF0.woff2) format('woff2');
      unicode-range: U+0400-045F, U+0490-0491

Создадим словарь в который будем записывать данный которые нам нужны по заданию. Названия, описание, рейтинг и сферу деятельности компаний, дату публикации, а также текст статей из Интернет-ресурсов

In [10]:
result_list = {'title': [], 'namecompany': [], 'description': [], 'rating': [], 'field': [], 'date': [], 'textpub': []}

### Алгоритм

Суть алгоритма заключается в переборе страниц, и переходе на "вложенные" страницы, то есть у нас есть основная страница https://habr.com/ru/all/ мы перебираем несколько стараниц с page1 до page10. На каждой странице есть статьи, записываем их в список, чтобы перейти по ним используем  _-i.a.get('href')-_  то есть берём значение из href этого заголовка. Далее находим классы элементов которые нам нужны, и записываем их в результат.

In [11]:
pagenum = 1
for i in range(10):
    url = 'https://www.kinopoisk.ru/reviews/type/comment/period/month/page/' + str(pagenum) + '/#list' # переход на ссылуку с определённым номером сраницы
    page = requests.get(url)
    soup = bs(page.text, 'html.parser')
    titles = soup.find_all('h2', class_='tm-title tm-title_h2')# получаем заголовки всех статей на этой странице
    for i in titles: 
        # переход на страницу статьи
        url = 'https://habr.com' + str(i.a.get('href')) 
        page = requests.get(url)
        soup = bs(page.text, 'html.parser')
        
        name_company = soup.find('a', class_='tm-company-snippet__title')# получаем название компаний
        desc_company = soup.find('div', class_='tm-company-snippet__description')# получаем описание компаний
        
        if (name_company is not None): #если на странице присутсвует компания
        
            result_list['title'].append(i.text) # записываем название статьи
            result_list['namecompany'].append(name_company.text) # записываем название компании
            result_list['description'].append(desc_company.text) # записываем описание компании
            
            datepub = soup.find('span', class_='tm-article-datetime-published') # находим дату публикаций
            result_list['date'].append(datepub.time['datetime'][0: 10]) # записываем дату публикаций
            
            # текст статьи
            try:
                textpub = soup.find('div', class_='article-formatted-body article-formatted-body article-formatted-body_version-2').get_text()
                textpub = textpub.replace('\n', ' ').replace('\t', ' ').replace('\xa0', ' ').replace('\u200e', ' ').replace('\r', ' ')
            except:
                textpub = soup.find('div', class_='article-formatted-body article-formatted-body article-formatted-body_version-1').get_text()
                textpub = textpub.replace('\n', ' ').replace('\t', ' ').replace('\xa0', ' ').replace('\u200e', ' ').replace('\r', ' ')
            result_list['textpub'].append(textpub)
            
            # переход на страницу компании
            url = 'https://habr.com' + str(name_company.get('href'))
            page = requests.get(url)
            soup = bs(page.text, 'html.parser')
            
            #записываем рейтинг
            rating = soup.find('span', class_='tm-votes-lever__score-counter tm-votes-lever__score-counter tm-votes-lever__score-counter_rating')
            if(rating is None):
                result_list['rating'].append('0')
            else:
                result_list['rating'].append((rating.text).strip())
               
             #записываем отрасли компаний
            fieldtext = ""
            fields = soup.find_all('a', 'tm-company-profile__categories-text')
            for field in fields:
                fieldtext = fieldtext + ((field.text).strip()) + ", "
            if (fields is None):
                result_list['field'].append(None)
            else:
                result_list['field'].append(fieldtext[0:-2])
            
    pagenum += 1

Что бы найти элемет на странице, выделите этот элемет(заголовок, текст, изображение) и нажмите Ctrl + Shift + I или ПКМ и "Исследовать элемент".

In [12]:
result_list

{'title': ['Новый сервер ВКС от Yealink — встречаем UC4X и новые лицензии',
  'Security Week 2404: подглядывание через датчик освещенности',
  'Когда был большой взрыв в виртуальной реальности? История развития VR-технологий',
  'Наследие Windows XP: загадка title.wma',
  'Книга «Гейм-дизайн: как создаются игры»',
  'Java ScopedValue: Ускоренный ThreadLocal',
  'Как в Индии судились с программистами: 5 примечательных кейсов последних лет',
  '«Это длилось годами». Внутри производственного бардака Boeing',
  'Новинки CES 2024',
  'Знакомство с Jest Mocks',
  'Кибер Бэкап: отказоустойчивость сервера управления',
  'Geniatech XPI-3566-Zero: что это за одноплатник и на что он способен',
  'Как я за один заход хакнул половину американских сетей фастфуда',
  'Золотая эпоха в микроэлектронике',
  'CodeLLM теперь в Поиске Brave',
  'Личный опыт: переход с Redux на Effector. И при чем тут DX',
  'Показалось, что ИИ уже ворвался в нашу жизнь, но нет',
  'Разработка онлайн-тестов для оценки профе

In [13]:
print("Количество нулевых значений в: ")
for i in result_list:
    print( i + " - " + str(result_list[i].count(None)))

Количество нулевых значений в: 
title - 0
namecompany - 0
description - 0
rating - 0
field - 0
date - 0
textpub - 0


### Сохранение данных

In [14]:
file_name = 'habr.csv'
df = pd.DataFrame(data=result_list)
df.to_csv(file_name)

In [15]:
df.head()

,title,namecompany,description,rating,field,date,textpub
0,Новый сервер ВКС от Yealink — встречаем UC4X и...,STSS,STSS — отечественный производитель и интегратор,42.23,"Программное обеспечение, Аппаратное обеспечени...",2024-01-22,Больше двух лет назад обновления Yealink Meeti...
1,Security Week 2404: подглядывание через датчик...,«Лаборатория Касперского»,"Ловим вирусы, исследуем угрозы, спасаем мир",202.45,Программное обеспечение,2024-01-22,Научные исследования в сфере безопасности дово...
2,Когда был большой взрыв в виртуальной реальнос...,Selectel,IT-инфраструктура для бизнеса,1555.08,"Аппаратное обеспечение, Связь и телекоммуникац...",2024-01-22,"Сегодня шлемы Oculus Rift, Kinect, гарнитуры..."
3,Наследие Windows XP: загадка title.wma,МТС,Экосистема цифровых сервисов,990.5,"Связь и телекоммуникации, Мобильные технологии...",2024-01-22,Старые операционные системы и приложения до си...
4,Книга «Гейм-дизайн: как создаются игры»,Издательский дом «Питер»,Компания,158.39,"СМИ, Электронная коммерция, Производство мульт...",2024-01-22,"Привет, Хаброжители! Узнайте, как придумать..."


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77 entries, 0 to 76
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        77 non-null     object
 1   namecompany  77 non-null     object
 2   description  77 non-null     object
 3   rating       77 non-null     object
 4   field        77 non-null     object
 5   date         77 non-null     object
 6   textpub      77 non-null     object
dtypes: object(7)
memory usage: 4.3+ KB


Итого у нас получилось собрать данные с 10ти страниц, нашли 88 статей в которых присутсвует компания.